# Evaluating linear and polynomial regression with elastic net regularization

In [1]:
import pandas
import numpy as np
from sklearn import preprocessing
from scipy import stats
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import KFold
import sklearn.linear_model as lm
import random
import DataPrepUtil
import Impute

## Reading data

In [2]:
housing = pandas.read_csv('./housing.csv')

## Preprocessing data

In [3]:
DataPrepUtil.transform_ocean_proximity(housing)
Impute.fill_lr_prediction_from_other_column(housing, 'total_rooms')

standard = preprocessing.StandardScaler().fit(housing)
df = standard.transform(housing)

median_house_value_bc, maxlog, interval = stats.boxcox(housing.median_house_value, alpha=0.05)
population_bc, maxlog, interval = stats.boxcox(housing.population, alpha=0.05)
housing_median_age_bc, maxlog, interval = stats.boxcox(housing.housing_median_age, alpha=0.05)
total_rooms_bc, maxlog, interval = stats.boxcox(housing.total_rooms, alpha=0.05)
total_bedrooms_bc, maxlog, interval = stats.boxcox(housing.total_bedrooms, alpha=0.05)
households_bc, maxlog, interval = stats.boxcox(housing.households, alpha=0.05)
median_income_bc, maxlog, interval = stats.boxcox(housing.median_income, alpha=0.05)

housing_boxcox = housing.copy()

housing_boxcox.drop(columns=['housing_median_age'], inplace=True)
housing_boxcox.drop(columns=['total_rooms'], inplace=True)
housing_boxcox.drop(columns=['total_bedrooms'], inplace=True)
housing_boxcox.drop(columns=['population'], inplace=True)
housing_boxcox.drop(columns=['households'], inplace=True)
housing_boxcox.drop(columns=['median_income'], inplace=True)
housing_boxcox.drop(columns=['median_house_value'], inplace=True)

housing_boxcox['housing_median_age'] = housing_median_age_bc
housing_boxcox['total_rooms'] = total_rooms_bc
housing_boxcox['total_bedrooms'] = total_bedrooms_bc
housing_boxcox['population'] = population_bc
housing_boxcox['households'] = households_bc
housing_boxcox['median_income'] = median_income_bc
housing_boxcox['median_house_value'] = median_house_value_bc

c:\python37\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
c:\python37\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
c:\python37\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """


## Splitting input and output

In [4]:
y = housing_boxcox.median_house_value.values.reshape(-1,1)
X = housing_boxcox.drop(columns=['median_house_value'], inplace=False).values

## Create polynomial features from the original input features (for polynomial regression)

In [5]:
poly = PolynomialFeatures(degree=3)
X_poly = poly.fit_transform(housing_boxcox.drop(columns=['median_house_value'], inplace=False).values)
print(X_poly)
print(X_poly.shape)

[[ 1.00000000e+00 -1.22230000e+02  3.78800000e+01 ...  2.00602423e+02
   5.06028832e+01  1.27648098e+01]
 [ 1.00000000e+00 -1.22220000e+02  3.78600000e+01 ...  8.24729756e+02
   1.02375247e+02  1.27080308e+01]
 [ 1.00000000e+00 -1.22240000e+02  3.78500000e+01 ...  2.35946960e+02
   4.91591188e+01  1.02422127e+01]
 ...
 [ 1.00000000e+00 -1.21220000e+02  3.94300000e+01 ...  1.06200069e+02
   4.13060842e+00  1.60658331e-01]
 [ 1.00000000e+00 -1.21320000e+02  3.94300000e+01 ...  1.09380005e+02
   5.38606468e+00  2.65219339e-01]
 [ 1.00000000e+00 -1.21240000e+02  3.93700000e+01 ...  2.00983807e+02
   1.22271651e+01  7.43858764e-01]]
(20640, 560)


## Training and test set split

In [6]:
test = random.sample(range(0,20640),5000)
X_test = X[test]
X_poly_test = X_poly[test]
y_test = y[test]
X_train_valid = np.delete(X, test, 0)
X_poly_train_valid = np.delete(X_poly, test, 0)
y_train_valid = np.delete(y, test, 0)

## Variables for holding the optimal models and the corresponding validation score

In [7]:
ChosenLinearModel = None
current_score_linear = -1
current_paramater_linear = -1

ChosenPolyModel = None
current_score_poly = -1
current_paramater_poly = -1

## Performance of linear regression on different value of regularization parameter

In [8]:
for a in np.arange(0.01, 0.2, 0.02):
    print("a = " + str(a))
    
    Model = lm.ElasticNet(alpha=a, max_iter=3000, tol=0.13)
    
    # Have to shuffle the data because it is grouped.
    kf = KFold(n_splits=5, shuffle=True)
    total_train_score = 0
    total_valid_score = 0
    
    for train_index, valid_index in kf.split(X_train_valid):
        X_train, X_valid = X_train_valid[train_index], X_train_valid[valid_index]
        y_train, y_valid = y_train_valid[train_index], y_train_valid[valid_index]
        Model.fit(X_train, y_train)
        total_train_score += Model.score(X_train, y_train)
        total_valid_score += Model.score(X_valid, y_valid)
        
    avg_train_score = total_train_score / 5
    avg_valid_score = total_valid_score / 5
    avg_train_valid_score = (avg_train_score + avg_valid_score) / 2
    print("Average training r2 score: " + str(avg_train_score))
    print("Average validation r2 score: " + str(avg_valid_score))
    
    if (ChosenLinearModel is None) or (avg_train_valid_score > current_score_linear):
        ChosenLinearModel = Model
        current_score_linear = avg_train_valid_score
        current_paramater_linear = a

print("Optimal model (linear): a = " + str(current_paramater_linear))

a = 0.01
Average training r2 score: 0.6885423215396218
Average validation r2 score: 0.687406395661195
a = 0.03
Average training r2 score: 0.6801704017046205
Average validation r2 score: 0.6791626860594038
a = 0.049999999999999996
Average training r2 score: 0.6739895040423836
Average validation r2 score: 0.6733343051136194
a = 0.06999999999999999
Average training r2 score: 0.6673287783136242
Average validation r2 score: 0.6665812867824135
a = 0.08999999999999998
Average training r2 score: 0.6601728825412453
Average validation r2 score: 0.6593942788187006
a = 0.10999999999999997
Average training r2 score: 0.6509657531391504
Average validation r2 score: 0.6502667037412708
a = 0.12999999999999998
Average training r2 score: 0.6434000010589956
Average validation r2 score: 0.6430455729684368
a = 0.15
Average training r2 score: 0.6321182408748957
Average validation r2 score: 0.6317212969155326
a = 0.16999999999999998
Average training r2 score: 0.6234861136019806
Average validation r2 score: 0.

## Performance of polynomial regression on different value of regularization parameter

In [9]:
for a in np.arange(0.01, 100, 10):
    print("a = " + str(a))
    
    Model = lm.ElasticNet(alpha=a, max_iter=3000, tol=0.13)
    
    # Have to shuffle the data because it is grouped.
    kf = KFold(n_splits=5, shuffle=True)
    total_train_score = 0
    total_valid_score = 0
    
    for train_index, valid_index in kf.split(X_train_valid):
        X_train, X_valid = X_poly_train_valid[train_index], X_poly_train_valid[valid_index]
        y_train, y_valid = y_train_valid[train_index], y_train_valid[valid_index]
        Model.fit(X_train, y_train)
        total_train_score += Model.score(X_train, y_train)
        total_valid_score += Model.score(X_valid, y_valid)
        
    avg_train_score = total_train_score / 5
    avg_valid_score = total_valid_score / 5
    avg_train_valid_score = (avg_train_score + avg_valid_score) / 2
    print("Average training r2 score: " + str(avg_train_score))
    print("Average validation r2 score: " + str(avg_valid_score))
    
    if (ChosenPolyModel is None) or (avg_train_valid_score > current_score_poly):
        ChosenPolyModel = Model
        current_score_poly = avg_train_valid_score
        current_paramater_poly = a

print("Optimal model (polynomial): a = " + str(current_paramater_poly))

a = 0.01
Average training r2 score: 0.7432349724072532
Average validation r2 score: 0.7389678610621365
a = 10.01
Average training r2 score: 0.7346471896612148
Average validation r2 score: 0.7320860652606302
a = 20.01
Average training r2 score: 0.7278216882240824
Average validation r2 score: 0.7256346944606961
a = 30.01
Average training r2 score: 0.7226073943492676
Average validation r2 score: 0.7207564821015982
a = 40.01
Average training r2 score: 0.718943798548835
Average validation r2 score: 0.716840083082827
a = 50.01
Average training r2 score: 0.7158976662402797
Average validation r2 score: 0.714200531804442
a = 60.01
Average training r2 score: 0.7130240391392585
Average validation r2 score: 0.710805698198589
a = 70.01
Average training r2 score: 0.7102482573690845
Average validation r2 score: 0.7088962074681094
a = 80.01
Average training r2 score: 0.7081271983121876
Average validation r2 score: 0.7055945889911082
a = 90.01
Average training r2 score: 0.7051894555707398
Average valid

## Final performance score of the optimal models evaluated on test set

In [10]:
print("R2 score of optimal linear model: " + str (ChosenLinearModel.score(X_test, y_test)))
print("R2 score of optimal polynomial model: " + str(ChosenPolyModel.score(X_poly_test, y_test)))

R2 score of optimal linear model: 0.6860348218142982
R2 score of optimal polynomial model: 0.742368019780205
